In [1]:
import sys
sys.path.append('..')

In [2]:
import modules.initialize as minit

minit.initialize(verbose=True)

Detected CUDA files, patching ldflags
Emitting ninja build file /home/alexey/torch_experiments/mytorchcudamodules/notebooks/../csrc/build/build.ninja...
Building extension module fused_mix_prec_layer_norm_cuda...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module fused_mix_prec_layer_norm_cuda...


In [3]:
from modules import SelfMultiheadAttn, EncdecMultiheadAttn
import torch

In [4]:
device = torch.device('cuda')

In [5]:
seq_length   = 80
sequences    = 10
hidden_dim   = 1024
heads        = 16
dropout_prob = 0.0

In [6]:
tst_layer = EncdecMultiheadAttn(hidden_dim, 
                               heads, 
                               dropout=dropout_prob, 
                               bias=False, 
                               include_norm_add=False, 
                               impl='default')
tst_layer = tst_layer.to(device)

In [7]:

tst_inputs_q = torch.randn(seq_length, sequences, hidden_dim, 
                                device=torch.device("cuda")).requires_grad_(True)
tst_inputs_k = torch.randn(seq_length, sequences, hidden_dim, 
                                device=torch.device("cuda")).requires_grad_(True)

tst_outputs,_ = tst_layer.forward(tst_inputs_q, 
                                   tst_inputs_k, 
                                   tst_inputs_k,
                                   key_padding_mask=None, 
                                   need_weights=False, 
                                   attn_mask=None,
                                   is_training=True)

#print(tst_outputs.shape)

tst_inputs_q.backward(torch.randn_like(tst_inputs_q))